In [1]:
from keras import Sequential
from keras.layers import LSTM, Dense
from build_train_set import buildTrainSet, makePvBinary
import numpy as np
import os

In [2]:
# run sekali aja (atau kalo buildTrainSet ada yang diubah)
freq, pv = buildTrainSet()

# write to .txt file (biar gausah build train set berkali2)
cur_path = os.path.abspath('')
write_path = os.path.join(cur_path, '../train')

freq_file = open(os.path.join(write_path, 'freqs.txt'), 'w')
for f in freq:
    freq_file.write(str(f)+'\n')
freq_file.close()

pv_file = open(os.path.join(write_path, 'pvs.txt'), 'w')
for p in pv:
    pv_file.write(str(p)+'\n')
pv_file.close()

8/8 [==============================] - 10s 1s/step


In [2]:
# read train files
cur_path = os.path.abspath('')
read_path = os.path.join(cur_path, '../train')

freq_f = open(os.path.join(read_path, 'freqs.txt'), 'r')
freq = [float(line.rstrip('\n')) for line in freq_f]
freq_f.close()

pv_f = open(os.path.join(read_path, 'pvs.txt'), 'r')
pv = [int(line.rstrip('\n')) for line in pv_f]
pv_f.close()

In [3]:
# temp freq converter
freq_new = []
i = 0

while (i < 1107750):
    freq_new.append(freq[i:i+250])
    i += 250

freq = np.array(freq_new).reshape((4431, 250, 1))
freq.shape

(4431, 250, 1)

In [4]:
# temp pv converter
pv_new = []
i = 0

while (i < 1107750):
    pv_new.append(pv[i:i+250])
    i += 250

pv = makePvBinary(pv_new)
pv.shape

(4431, 250, 58)

In [5]:
# Split datasets
# 80% train, 10% test, 10% tune
split_train_test = int(len(pv) / 10 * 8)
split_test_tune = int(len(pv) / 10 * 9)

x_train = freq[0:split_train_test]
y_train = pv[0:split_train_test]

x_test = freq[split_train_test:split_test_tune]
y_test = pv[split_train_test:split_test_tune]

x_tune = freq[split_test_tune:len(pv)]
y_tune = pv[split_test_tune:len(pv)]

print((x_train.shape, y_train.shape), (x_test.shape, y_test.shape), (x_tune.shape, y_tune.shape))

((3544, 250, 1), (3544, 250, 58)) ((443, 250, 1), (443, 250, 58)) ((444, 250, 1), (444, 250, 58))


In [6]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(250,1)))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(Dense(58, activation='softmax')) # range 30-86 + 0

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 250, 256)          264192    
                                                                 
 lstm_1 (LSTM)               (None, 250, 256)          525312    
                                                                 
 lstm_2 (LSTM)               (None, 250, 256)          525312    
                                                                 
 dense (Dense)               (None, 250, 58)           14906     
                                                                 
Total params: 1,329,722
Trainable params: 1,329,722
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
model.fit(x_train, y_train, epochs=20, batch_size=300)

Epoch 1/20
12/12 [==============================] - 290s 24s/step - loss: 3.0843 - acc: 0.2855
Epoch 2/20
12/12 [==============================] - 272s 24s/step - loss: 2.3921 - acc: 0.3570
Epoch 3/20
12/12 [==============================] - 273s 23s/step - loss: 1.9731 - acc: 0.4073
Epoch 4/20
12/12 [==============================] - 279s 24s/step - loss: 1.6256 - acc: 0.4851
Epoch 5/20
12/12 [==============================] - 270s 23s/step - loss: 1.3939 - acc: 0.5473
Epoch 6/20
12/12 [==============================] - 277s 24s/step - loss: 1.2249 - acc: 0.6060
Epoch 7/20
12/12 [==============================] - 281s 24s/step - loss: 1.1152 - acc: 0.6334
Epoch 8/20
12/12 [==============================] - 285s 25s/step - loss: 1.0086 - acc: 0.6845
Epoch 9/20
12/12 [==============================] - 280s 24s/step - loss: 0.9233 - acc: 0.7220
Epoch 10/20
12/12 [==============================] - 291s 25s/step - loss: 0.8625 - acc: 0.7376
Epoch 11/20
12/12 [==============================

In [8]:
accr = model.evaluate(x_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

14/14 [==============================] - 37s 1s/step - loss: 0.5868 - acc: 0.8203
Test set
  Loss: 0.587
  Accuracy: 0.820
